# Get a Biomass function using a GSM

## Kugler2023
**Model Publication:**  
Kugler, A., Stensjö, K. Optimal energy and redox metabolism in the cyanobacterium Synechocystis sp. PCC 6803. npj Syst Biol Appl 9, 47 (2023). https://doi.org/10.1038/s41540-023-00307-3

**Available at:**  
https://github.com/amitkugler/CBA

## Knoop2015
**Model Publication:**  
Knoop H and Steuer R (2015) A computational analysis of stoichiometric constraints and trade-offs in cyanobacterial biofuel production. Front. Bioeng. Biotechnol. 3:47. doi: 10.3389/fbioe.2015.00047

**Available at:**  
https://www.frontiersin.org/articles/file/downloadfile/136766_supplementary-materials_datasheets_1_zip/octet-stream/Data%20Sheet%201.ZIP/1/136766?isPublishedV2=False

In [21]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import os
from functools import partial
from concurrent.futures import ProcessPoolExecutor
from tqdm.auto import tqdm
import warnings

from cobra.io import load_json_model, read_sbml_model
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba

from cobra import Model, Reaction, Metabolite
from cobra.manipulation.validate import check_mass_balance

from modelbase.ode import Simulator
from modelbase.ode import ratefunctions as rf

sys.path.append("../Code")
import functions as fnc
import calculate_parameters_restruct as prm
import functions_light_absorption as lip

# Import model functions
from get_current_model import get_model

from functions_custom_steady_state_simulator import simulate_to_steady_state_custom

In [9]:
select_model = "Kugler2023"
# select_model = "Knoop2015"
# select_model = "Knoop2013"

In [10]:
def add_import(model, metabolite_name, reaction_stoichiometry, bounds=(-1000,1000)):
    # Define the reaction
    reac = Reaction(f'{metabolite_name}_import')
    reac.name = f'{metabolite_name} import'
    reac.bounds = bounds

    # Add the correct stoichiometry and add to the model
    reac.add_metabolites({model.metabolites.get_by_id(k): v for k,v in reaction_stoichiometry.items()})
    model.add_reactions([reac])
    
    return model

In [11]:
def get_stoichiometric_fluxes(model, solution, compound):
    # Get the reactions where the compound is involved and their stoichiometry
    react = solution.fluxes[[x.id for x in model.metabolites.get_by_id(compound).reactions]]
    stoich = pd.Series({x:model.reactions.get_by_id(x).metabolites[model.metabolites.get_by_id(compound)] for x in react.index})
    
    # Return the scaled flux according to the stoichiometry
    return react * stoich

In [12]:
# Load the model
if select_model == "Kugler2023":
    model = load_json_model('../GSM_models/Kugler2023/GSM/JSON/iJN678_AK.json')

    # Set phototrophic growth and modify the model as described in the Analysis by Kugler
    #set model objective to autotrophic growth
    model.objective = 'BIOMASS_Ec_SynAuto'
    #set photoautotropic growth by constraining glucose exchange reatcion
    model.reactions.EX_glc__D_e.bounds = (0, 1000)
    #set photoautotropic growth by constraining HCO3 exchange reatcion.
    model.reactions.EX_co2_e.bounds = (0, 1000)
    model.reactions.EX_hco3_e.bounds = (-3.7, 1000)
    #set photon flux to lab scale conditions.
    model.reactions.EX_photon_e.bounds = (-45, -45)
    #blocking transhydrogenase PntAB
    model.reactions.NADTRHD.bounds = (0, 0)

    # Make the model loopless (makes model infeasible)
    # add_loopless(model)

elif select_model == "Knoop2015":
    model = read_sbml_model("../GSM_models/Knoop2015/Knoop2015_Supplement_revised/Metabolic_Network_supplementary_File1.xml")

    # Constraints used for the FBA simulation
    # Photon input
    model.reactions.PR0001.bounds = (0,15.57)

elif select_model == "Knoop2013":
    model = read_sbml_model("../GSM_models/Knoop2013/pcbi.1003081.s002.xml")

In [13]:
with model:
    sol_loopless = loopless_solution(model)

In [14]:
solutions = {}
#############
# Get the default flux
solutions["default"] = model.optimize("maximize")
solutions["default_pfba"] = pfba(model)

In [16]:
model.summary(solutions["default"])

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0003718,0,0.00%
cobalt2_e,EX_cobalt2_e,0.000268,0,0.00%
cu2_e,EX_cu2_e,0.0002479,0,0.00%
fe2_e,EX_fe2_e,0.0006168,0,0.00%
fe3_e,EX_fe3_e,0.0005616,0,0.00%
h_e,EX_h_e,4.071,0,0.00%
hco3_e,EX_hco3_e,3.451,1,99.99%
k_e,EX_k_e,0.01394,0,0.00%
mg2_e,EX_mg2_e,0.00237,0,0.00%
mn2_e,EX_mn2_e,0.0002496,0,0.00%


In [19]:

with model:
    ##############
    # With imports
    # Disable light reactions
    model.reactions.EX_photon_e.bounds = (0, 0)

    # Disable NDH-1 as we want to measure ATP import
    # model.reactions.get_by_id("NDH1_1u").bounds = (0, 0)
    # model.reactions.get_by_id('NDH1_2u').bounds = (0, 0)
    # model.reactions.get_by_id('NDH1_3u').bounds = (0, 0)
    # model.reactions.get_by_id('NDH1_4pp').bounds = (0, 0)
    

    # # Disable Terminal oxidases
    model.reactions.get_by_id('CYO1b_syn').bounds = (0, 0) # COX
    model.reactions.get_by_id("CYTBDu").bounds = (0, 0) # Cyd
    model.reactions.get_by_id("CYTBDpp_1").bounds = (0, 0) # Cyd
    model.reactions.get_by_id('Flv2_4').bounds = (0, 0) # Flv2/4
    model.reactions.get_by_id('ARTO').bounds = (0, 0) # ARTO

    # # Disable H2 production
    # model.reactions.get_by_id("H2ASE_syn").bounds = (0, 0)

    # # Disable Cyt b6f
    # model.reactions.get_by_id('CBFC2').bounds = (0, 0)

    # # Disable the Mehler reaction
    model.reactions.get_by_id("MEHLER").bounds = (0, 0)

    # # model.reactions.get_by_id("ATPS4rpp_1").bounds = (0,0)
    # model.reactions.get_by_id("ATPSu").bounds = (0,0)

    # Instead, add Exchange reactions for ATP, NADPH and 3PGA
    pseudo_reactions = {
        "3PGA":  ({"3pg_c":1}, (-1, 1)),
        "ATP":   ({"atp_c":1, "adp_c":-1}, (0, 1000)),
        "NADPH": ({"nadph_c":1, "h_c":1, "nadp_c":-1}, (0, 1000)),
        "H+": ({"h_c":1}, (-1000, 1000)),
    }
    for name, (stoich, bounds) in pseudo_reactions.items():
        model = add_import(model, name, stoich, bounds)

    # Set a new objective function
    model.objective = {
        model.reactions.get_by_id('BIOMASS_Ec_SynAuto'):0.9,
        model.reactions.get_by_id('ATP_import'):0.1,
    
    }

    # Disable other Carbon import reactions
    # HCO3 import
    # model.reactions.EX_hco3_e.lower_bound = 0

    solutions["imports"] = model.optimize("maximize")
    solutions["imports_pfba"] = pfba(model)

    print(model.summary(solutions["imports"]))

# Print a summary of the models
for nam, sol in solutions.items():
    print(f"""
model: {nam}
biomass: {sol.fluxes.get("BIOMASS_Ec_SynAuto")}
ATP: {sol.fluxes.get("ATP_import")}
NADPH: {sol.fluxes.get("NADPH_import")}
3PGA: {sol.fluxes.get("3PGA_import")}
""")

Objective
0.9 BIOMASS_Ec_SynAuto + 0.1 ATP_import = 0.11619772353098551

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
     ca2_e     EX_ca2_e 7.542E-05         0   0.00%
 cobalt2_e EX_cobalt2_e 5.436E-05         0   0.00%
     cu2_e     EX_cu2_e 5.028E-05         0   0.00%
     fe2_e     EX_fe2_e 0.0001251         0   0.00%
     fe3_e     EX_fe3_e 0.0001139         0   0.00%
       h_e       EX_h_e     51.67         0   0.00%
    hco3_e    EX_hco3_e       3.7         1 100.00%
       k_e       EX_k_e  0.002828         0   0.00%
     mg2_e     EX_mg2_e 0.0004808         0   0.00%
     mn2_e     EX_mn2_e 5.063E-05         0   0.00%
    mobd_e    EX_mobd_e 5.064E-05         0   0.00%
     na1_e     EX_na1_e  6.33E-05         0   0.00%
     no3_e     EX_no3_e    0.1507         0   0.00%
      o2_e      EX_o2_e  0.009053         0   0.00%
     so4_e     EX_so4_e  0.003171         0   0.00%
     zn2_e     EX_zn2_e 5.028E-05         0   0.00%
  rdmbzi_c  SK_rdmbzi_c 3.728